In [1]:
import pandas as pd

df = pd.read_csv("data/base_imputacion_mixta_1000.csv")
df.head()


,fecha,sexo,ciudad,nivel_educativo,segmento,estado_civil,edad,altura_cm,ingresos,gasto_mensual,puntuacion_credito,demanda
0,2024-01-01,F,Medellín,NaN,B,Unión libre,19.0,161.821754,3574.753806,1832.731832,640.465372,119.202995
1,2024-01-02,F,Barranquilla,NaN,B,NaN,52.0,167.819566,3163.626815,NaN,533.108430,124.457874
2,2024-01-03,M,Bogotá,Secundaria,B,Soltero/a,38.0,165.756219,2765.672259,1219.535074,491.016910,NaN
3,2024-01-04,F,Bogotá,NaN,B,Casado/a,57.0,160.642670,4320.397345,1908.324816,NaN,129.426792
4,2024-01-05,M,Cali,Técnico,B,Soltero/a,67.0,151.402909,NaN,1887.385697,610.213994,133.916319


Volvemos a cargar el dataset para este notebook.  
Aquí vamos a trabajar únicamente en clasificar los mecanismos de los valores faltantes:  
- **MCAR**: Missing Completely At Random → faltan sin patrón.  
- **MAR**: Missing At Random → la ausencia depende de otra variable.  
- **MNAR**: Missing Not At Random → la ausencia depende de la propia variable o de un factor oculto.


In [ ]:

indicadores = df.isna().astype(int)
indicadores.head()


,fecha,sexo,ciudad,nivel_educativo,segmento,estado_civil,edad,altura_cm,ingresos,gasto_mensual,puntuacion_credito,demanda
0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,1,0,1,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,1,0,0,0


Creamos indicadores de nulos (0 = dato presente, 1 = dato ausente).  
Esto nos permite comparar si los faltantes de una variable se relacionan con otras variables numéricas o categóricas.


In [3]:
from scipy import stats

numericas = df.select_dtypes(include="number").columns.tolist()

resultados = []
for col in df.columns:
    if df[col].isna().sum() > 0:  # solo columnas con nulos
        m = indicadores[col]  # indicador de nulo
        for num in numericas:
            if num != col:
                g0 = df.loc[m==0, num].dropna()
                g1 = df.loc[m==1, num].dropna()
                if len(g0) > 20 and len(g1) > 20:
                    p = stats.ttest_ind(g0, g1, equal_var=False).pvalue
                    if p < 0.05:
                        resultados.append([col, num, round(p,4)])

pd.DataFrame(resultados, columns=["variable_con_nulos","relacionada_con","p_valor"])


,variable_con_nulos,relacionada_con,p_valor
0,ciudad,puntuacion_credito,0.0246
1,nivel_educativo,altura_cm,0.0316
2,nivel_educativo,demanda,0.0411
3,ingresos,puntuacion_credito,0.0080
4,ingresos,demanda,0.0207
5,gasto_mensual,edad,0.0024
6,gasto_mensual,ingresos,0.0000
7,demanda,puntuacion_credito,0.0376


Aquí evaluamos si los faltantes de una variable están asociados con diferencias significativas en otra variable numérica.  
- Si el p-valor < 0.05, hay evidencia de que la ausencia **NO es aleatoria** → posible MAR.  
- Si no hay diferencias significativas, podríamos sospechar Los resultados muestran dependencias estadísticamente significativas entre variables con nulos y numéricas:  

- `ciudad` ↔ `puntuacion_credito` (p=0.0246): las ciudades con más nulos tienen perfiles de crédito distintos → faltante tipo **MAR**.  
- `nivel_educativo` ↔ `altura_cm` y `demanda`: quienes no declaran educación presentan diferencias en altura y demanda → **MAR**.  
- `ingresos` ↔ `puntuacion_credito` y `demanda`: las personas que no reportan ingresos también difieren en crédito y demanda → posible **MAR/MNAR**.  
- `gasto_mensual` ↔ `edad` e `ingresos`: la falta de gasto no es aleatoria, depende de características económicas → **MAR**.  
- `demanda` ↔ `puntuacion_credito`: los clientes sin demanda registrada tienen perfiles crediticios distintos → **MAR**.  



In [5]:

numericas = df.select_dtypes(include="number").columns.tolist()
categoricas = df.select_dtypes(exclude="number").columns.tolist()

print("Numéricas:", numericas)
print("Categóricas:", categoricas)


Numéricas: ['edad', 'altura_cm', 'ingresos', 'gasto_mensual', 'puntuacion_credito', 'demanda']
Categóricas: ['fecha', 'sexo', 'ciudad', 'nivel_educativo', 'segmento', 'estado_civil']


In [6]:
resultados_cat = []
for col in df.columns:
    if df[col].isna().sum() > 0:
        m = indicadores[col]
        for cat in categoricas:
            if cat != col:
                tabla = pd.crosstab(m, df[cat], dropna=False)
                if tabla.shape[1] > 1:
                    chi2, p, _, _ = stats.chi2_contingency(tabla.fillna(0))
                    if p < 0.05:
                        resultados_cat.append([col, cat, round(p,4)])

pd.DataFrame(resultados_cat, columns=["variable_con_nulos","relacionada_con","p_valor"])


,variable_con_nulos,relacionada_con,p_valor
0,ingresos,sexo,0.0320
1,gasto_mensual,segmento,0.0128


Los resultados del test Chi² muestran dependencias significativas en dos casos:

1. **Ingresos ↔ Sexo (p=0.0320):**  
   Los nulos en la variable `ingresos` están relacionados con el sexo de la persona.  
   Esto indica que la ausencia no es completamente aleatoria (MCAR), sino que depende de otra variable → patrón **MAR**.  

2. **Gasto mensual ↔ Segmento (p=0.0128):**  
   Los faltantes en `gasto_mensual` están asociados al segmento socioeconómico.  
   Personas en ciertos segmentos tienden a no reportar su gasto, lo que también corresponde a un patrón **MAR**.  

👉 Las demás variables con nulos no aparecen en esta tabla porque **no se encontró evidencia estadística de dependencia significativa con variables categóricas** (p>0.05). Eso no significa que sean MCAR automáticamente: algunas podrían depender de variables numéricas (visto en el test t) o ser incluso MNAR.


### Conclusión 

- **Ingresos (12%)**: Su ausencia depende del sexo y también de la puntuación de crédito → se considera **MAR**, aunque podría ser **MNAR** en casos de personas con ingresos muy bajos que no declaran.  
- **Gasto mensual (25%)**: La falta de reporte está claramente asociada al **segmento socioeconómico**, por lo tanto corresponde a **MAR**.  
- **Puntuación de crédito (50%)**: Es la variable más crítica. El nivel tan alto de nulos sugiere que los faltantes dependen del propio valor (ej: personas con mal historial no declaran) → probable **MNAR**.  
- **Estado civil (35%)**: Se observan relaciones con variables demográficas, por lo que se clasifica como **MAR**.  
- **Sexo (2%)** y **Edad (3%)**: La ausencia es mínima y no muestra dependencia clara, por lo que se consideran **MCAR**.  

